# Interaction test

In [1]:
from openai import OpenAI
from dotenv import load_dotenv
import sys
import os

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [2]:
def call_openai(openai_client, messages, json_mode=False, temperature=0.7, model = "gpt-4.1-mini"):
    response = openai_client.chat.completions.create(
        model=model, # "gpt-4.1"
        messages=messages,
        temperature=temperature,
        response_format={"type": "json_object"} if json_mode else None
    )
    return response.choices[0].message.content.strip()

In [3]:
system_prompt = """
role: |
  You are a confident but mistake-prone coding student. You work with a peer who is teaching you to code. You collaborate to solve programming exercises by alternating rounds. In each round, the user works on their code and then shares their questions or comments, then you respond with your own changes, comments, and questions. Both you and the user have separate code panels, and you can see each other's code and outputs at all times.

interaction_protocol: |
  - You work in rounds: first the user edits and talks, then you edit and talk.
  - You always know:
    - the current user code
    - the current agent code
    - the user code's output or errors
    - the agent code's output or errors
  - These values are given to you in every conversation turn as global variables.
  - You must always consider these variables before proposing any solution.
  - You may freely read, comment on, and learn from the user's code panel, but your own edits happen in your own agent code panel.
  - Your Notebook is your memory of what the user has taught you so far. You can use the Notebook to solve problems, but no external knowledge beyond that.
  - You will update your Notebook by summarizing what the user explains to you.

personality: |
  You are confident, quick to jump in, and a little reckless. You use casual, short, playful language. When you realize a mistake, you might say things like “oops,” “oh shit,” or “dang.” You keep your answers brief so you do not crowd the chat window. You should sound funny, informal, and encouraging to keep the user engaged.

knowledge: |
  - You have partial understanding of basic code syntax and logic.
  - You do NOT know advanced techniques, libraries, or best practices unless taught by the user.
  - You do NOT know subtle language features or tricky algorithms unless explicitly explained by the user.
  - Your Notebook is your only growing knowledge. You cannot use outside knowledge from the internet or any other data.

notebook_instructions: |
  After each explanation from the user, you should confirm what you learned, summarize it, and add it to your Notebook. Use only your Notebook to reason in future rounds.

global_state_variables: |
  You will be given the following global variables on every turn:
    - user_code: the current code in the user's editor
    - agent_code: your current code
    - user_output: what happened when the user ran their code (including errors)
    - agent_output: what happened when you ran your code (including errors)
  Always read these carefully before starting your reasoning.
"""

In [4]:
# GLOBAL VARIABLES

problem_statement = """
# Problem Statement
## Description
**Nimm** is an ancient game of strategy that is named after the old German word for "take". It is also called *Tiouk Tiouk* in West Africa and *Tsynshidzi* in China. Players alternate taking stones until there are zero left. The game of Nimm goes as follows:

1. The game starts with a pile of 20 stones between the players
2. The two players alternate turns
3. On a given turn, a player may take either 1 or 2 stones from the center pile

The two players continue until the center pile has run out of stones.

The last player to take a stone loses.

> We have broken the problem down into three smaller milestones so you can build the solution step by step. Let's start with the first milestone.

## Task

**Milestone 1**

You'll write a `loop` to remove stones from a pile.

1. **Start with 20 stones.**

2. **Repeat this process until there are zero stones left:**
    - Print how many stones are left.  
      _Example:_ `There are 20 stones left`
    - Ask the user how many stones to remove — using this code:  
      ```python
      int(input("Would you like to remove 1 or 2 stones? "))
      ```
    - Subtract that number of stones from the total.
    - Add an empty `print()` between turns (to make the output easier to read).

3. **End the game when there are no stones left.**  
   _Print:_ `Game over`

**Important:**  
- For now, don't worry about whose turn it is.  
- Don't worry about making sure the user enters only 1 or 2 stones — any number is fine in this step.

**Example output:**  
```
There are 20 stones left
Would you like to remove 1 or 2 stones? 2

There are 18 stones left
Would you like to remove 1 or 2 stones? 17

There are 1 stones left
Would you like to remove 1 or 2 stones? 3

Game over
```
"""

user_code = ""
agent_code = ""
user_output = ""
agent_output = ""


In [5]:
messages = [
    {"role": "system", "content": system_prompt},
    # Global state variables
    {"role": "user", "content": f"""
    ```state
    [global_state]
     
    problem_statement: 
    {problem_statement}

    user_code:
    {user_code}

    agent_code:
    {agent_code}

    user_output:
    {user_output}

    agent_output:
    {agent_output}

    [/global_state]
    ```
    """
    }
]

user_message = [{"role": "user", "content": "OK, your turn! What would you like to do next?"}]

In [6]:
messages.extend(user_message)

In [8]:
response = call_openai(client, messages, json_mode = False, temperature = 0.7, model = "gpt-4.1-mini")

In [10]:
print(response)

Alright, I’ll take a stab at Milestone 1!

I’ll write a loop that starts with 20 stones, repeatedly asks how many stones to remove, subtracts that number, and stops when stones hit zero or below. I’ll print the messages exactly as the task says, including the empty line between turns.

Let me write the code now.
